Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from random import choices
from collections.abc import Callable, Sequence
import numpy as np

import lab9_lib
from population import Population
from population_builder import PopulationBuilder
from island import Island

In [ ]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

In [ ]:
def run_population(problem_fitness, generations, builder):
    population = (builder
                  .add_fitness_function(fitness_function=problem_fitness)
                  .build())
    population.run_for_generations_or_until_no_upgrades(generations, log_data=False, n_generations_without_upgrade=100)
    
    print(f"Generations ran {population.generations_ran}")
    print(f"Max fitness {population.max_fitness}")
    print(f"Fitness call {problem_fitness.calls / 1000}k")
    population.log_history_fitness()

# Population
Thanks, to the population builder, we can easily build a population, in this case after many test we develop this population that perform discretely in all problem( we the problem of premature convergence in the problems 5 and 10). 
Be aware that there are not Individual class but only a Population, so that every function applied to it is made with numpy function and not even a single for loop is present( this dramatically increase speed but reduce code readability). 

In [ ]:
#Population builder
builder = (PopulationBuilder()
           .initialize_random(population_size=100, genome_len=1000)
           .add_parents_selector_tournament(tournament_size=10, offspring_size=150)
           .add_survivals_selector_generational()
           .set_recombination_and_mutation_mutualexclusive(probability_recombination_over_mutation=.2)
           .add_mutation_single_flip()
           .add_recombination_uniform_xover())

In [ ]:
run_population(lab9_lib.make_problem(1), 1500, builder)

In [ ]:
run_population(lab9_lib.make_problem(2), 1500, builder)

In [ ]:
run_population(lab9_lib.make_problem(5), 1500, builder)

In [ ]:
run_population(lab9_lib.make_problem(10), 1500, builder)

# Islands
We have developed a solution base on island to try to mitigate the previous problem. 

In [ ]:
def run_island(population_size, genome_len, epoch, problem_fitness, builders ):
    builders = [b.add_fitness_function(fitness_function=problem_fitness) for b in builders]
    island = Island(population_size, genome_len, builders)
    
    island.run(epochs=epoch)
    print(f"Max fitness {island.max_fitness}")
    print(f"Fitness call {problem_fitness.calls / 1000}k")
    island.log_history_fitness()

In [ ]:
population_size = 50
genome_len = 1000
problem_fitness = lab9_lib.make_problem(2)

builders =[ (PopulationBuilder()
            .add_parents_selector_tournament(tournament_size=10, offspring_size=70)
            .add_survivals_selector_generational()
            .add_recombination_uniform_xover()
            .add_mutation_single_flip()
            .set_recombination_and_mutation_mutualexclusive(probability_recombination_over_mutation=0.2)),
            (PopulationBuilder()
            .add_parents_selector_tournament(tournament_size=2, offspring_size=70)
            .add_survivals_selector_generational()
            .add_recombination_one_point_xover()
            .add_mutation_single_flip()
            .set_recombination_and_mutation_mutualexclusive(probability_recombination_over_mutation=0.7)),
            (PopulationBuilder()
            .add_parents_selector_tournament(tournament_size=10, offspring_size=30)
            .add_survivals_selector_steady_state()
            .add_recombination_one_point_xover()
            .add_mutation_single_flip()
            .set_mutation_sequential_to_recombination(probability_mutation=0.3))]

In [ ]:
run_island(population_size, genome_len, 6, lab9_lib.make_problem(1), builders)

In [ ]:
run_island(population_size, genome_len, 20, lab9_lib.make_problem(2), builders)

In [ ]:
run_island(population_size, genome_len, 10, lab9_lib.make_problem(5), builders)

In [ ]:
run_island(population_size, genome_len, 10, lab9_lib.make_problem(10), builders)